# Test các tool cho việc extract entity, relation từ text

Ở đây mình thử dùng model [gpt-4o-mini] và framework llama_index để extract entity và relation từ text.

The choice between these extractors depends on the specific use case:

- Use SimpleLLMPathExtractor for exploratory analysis where you want to capture a wide range of potential relationships for RAG applications, without caring about the entity types.
- Use SchemaLLMPathExtractor when you have a well-defined domain and want to ensure consistency in the extracted knowledge.
- Use DynamicLLMPathExtractor when you want a balance between structure and flexibility, allowing the model to discover new entity and relation types while still providing some initial guidance. This one is especially useful if you want a KG with labeled (typed) entities but don't have an input Schema (or you've partially defined the schema as a starting base).


https://docs.llamaindex.ai/en/stable/examples/property_graph/Dynamic_KG_Extraction/

In [1]:
from dotenv import load_dotenv
import os
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

# Install the python-dotenv package if not already installed
%pip install python-dotenv

# Load the .env file

load_dotenv('../.env')

# Access the OpenAI key
openai_key = os.getenv("OPENAI_API_KEY")

llm = OpenAI(model="gpt-4o-mini", api_key=openai_key)
embed_model = OpenAIEmbedding(model="text-embedding-3-small")

Settings.llm = llm
Settings.embed_model = embed_model

FOLDER_DATA = "../data/bao-chi"

Note: you may need to restart the kernel to use updated packages.


In [7]:
import nest_asyncio

nest_asyncio.apply()

## Load tài liệu

In [35]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import PropertyGraphIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI

from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore

documents = SimpleDirectoryReader(FOLDER_DATA).load_data()
print("length of documents:", len(documents))
print('documents:', documents[21])



length of documents: 24
documents: Doc ID: 51cd37fa-0710-42b3-939e-cc29307b7941
Text: Kết Quả Kinh Doanh  **CTCP Trung tâm Hội chợ Triển lãm Việt Nam
(VEFAC – mã VEF)** vừa công bố báo cáo tài chính quý 3/2024 với các
chỉ số chính như sau:  - **Doanh thu thuần:** Chưa đến 4 tỷ đồng -
**Lợi nhuận gộp:** Gần 800 triệu đồng, khả quan hơn so với cùng kỳ năm
ngoái  Hoạt động tài chính tiếp tục là trụ cột gánh lợi nhuận của
doanh nghiệ...


In [22]:
!pip install pyvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 5.8 MB/s eta 0:00:00


In [38]:
from llama_index.core import Document, PropertyGraphIndex
from llama_index.core.indices.property_graph import DynamicLLMPathExtractor
# from config import entities, relations

entities = [
    "mã_cổ_phiếu", "công_ty", "lợi_nhuận", "giá_cổ_phiếu", "khối_lượng_giao_dịch", "ngày_giao_dịch",
    "tài_sản", "khoản_phải_thu", "dự_án", "doanh_thu", "lợi_nhuận_gộp", "lãi_ròng",
    "lợi_nhuận_sau_thuế", "tồn_kho", "khoản_phải_thu_ngắn_hạn", "bên_liên_quan", "ngành_công_nghiệp"
]

relations = [
    "sở_hữu", "thuộc_về", "là", "là_một_phần_của", "tăng", "giảm", "giao_dịch_vào_ngày",
    "công_bố", "triển_khai", "đạt", "phát_sinh", "trừ", "lên_đến", "chiếm", "hợp_tác", "liên_quan_tới"
]

allowed_entity_types = entities
allowed_relation_types = relations

kg_extractor = DynamicLLMPathExtractor(
    llm=llm, num_workers=4,
    max_triplets_per_chunk=20,
    allowed_entity_types=allowed_entity_types,
    allowed_relation_types=allowed_relation_types,
)

simple_index = PropertyGraphIndex.from_documents(
    documents,
    llm=llm,
    embed_kg_nodes=False,
    kg_extractors=[kg_extractor],
    show_progress=True,
)

Parsing nodes: 100%|██████████| 24/24 [00:00<00:00, 727.01it/s]
Extracting and inferring knowledge graph from text: 100%|██████████| 24/24 [00:34<00:00,  1.46s/it]


In [39]:
simple_index.property_graph_store.save_networkx_graph(
    name="./output/dynamicGraph1.html"
)


In [40]:
nodeSearch = simple_index.property_graph_store.get_triplets(
    entity_names=["VIC"]
)

print(nodeSearch)

[]


In [30]:
# Step 1: Extract node properties
nodes = simple_index.property_graph_store.get()
# print('nodes:', nodes)
node_ids = [node.id for node in nodes]
print('node_ids:', node_ids)

# Step 2: Get the label from each node's properties
node_labels = [node.label for node in nodes]
print('node_labels:', node_labels)
# node_properties = [" ".join([str(v) for v in node.properties.values()]) for node in nodes]
# print('node_properties:', node_properties)

node_ids: ['d11492ed-de3b-4ad8-a67b-8e4340f20932', '6031e1ca-a044-4db6-a672-68057ae9fe3e', '6050c6fa-117d-4da5-9289-7cf3e28fe0e1', '90bec3d1-06d6-4057-a384-69cc2f3c6f69', '77051f4d-ecbe-488f-8d4b-c48d80c63259', '5fa566e8-f6e4-45b5-a3e3-93f52792bd0a', '907d3f7c-bc2c-4771-89a7-c1a749a53b63', 'a61588cb-262d-4406-89f2-bd287f6a137a', '99a5e9a2-a0ac-403d-9a5a-8837fc8bd686', '47b00575-8288-41c7-9e75-0debdd9fdc1e', 'e57e5d93-ac0e-4e0a-8611-a86ccc5c6efa', '31b55d8b-cb73-4b90-a776-63b885334b0c', 'd3e01af5-8a27-4200-b6e9-46cac3aa132a', 'e5acb282-647f-4415-a687-ccdc31b2d6a1', 'c4b9a433-d851-40e7-ad6c-61632e7d0114', 'a66e9526-2c6d-4837-bee0-c2974a831d75', '90ebb549-477b-4698-ad82-eeb7fb34a4de', '9645ceed-630a-4c98-b1a6-730d390ae33e', '2dd8a003-baf4-4dc3-b423-dc5e41ae6229', 'f86ea768-e9a0-476d-ab2d-50b2dc9bae1d', '2aaa8adb-beb0-4b40-86b2-bfe90f3bc111', '1681d8f8-89a2-44ca-9f91-ac702ce22507', 'e82dd024-4448-4568-b5d0-1f6f864f4678', '6f0cc92e-4953-49bd-b4ef-54d97401bc1f', 'Bệnh viện Triều An', 'lợi nh

In [41]:
# use
retriever = simple_index.as_retriever(
    include_text=True,  # include source chunk with matching paths
    similarity_top_k=5,  # top k for vector kg node retrieval
)

question = "lãi sau thuế của công ty chứng khoán lpbank ?"
nodes = retriever.retrieve(question)
print("Retrieved nodes:")
for node in nodes:
    print(node.text)

print("Query engine:")
query_engine = simple_index.as_query_engine(
    include_text=True,  # include source chunk with matching paths
    similarity_top_k=5,  # top k for vector kg node retrieval
)
response = query_engine.query(question)
print("response:", response)


Retrieved nodes:
Query engine:
response: Empty Response


In [42]:
test_pairs = [
    ("lãi sau thuế của Chứng khoán LPBank ?", "Tăng 275% so với cùng kỳ năm trước"),
    ("thu gom cố phiếu MSN?", "Khối ngoại cũng giải ngân mua ròng 179 tỷ tại cổ phiếu MSN"),
    ("Doanh thu Bia Sài Gòn - Quảng Ngãi (BSQ) ?", "Bia Sài Gòn - Quảng Ngãi (BSQ) ghi nhận doanh thu 418 tỷ đồng"),
    ("Lực mua ròng các cổ phiếu nào ?", "ACV', 'YEG', 'TCB', 'TPB', 'MWG'"),
    ("VEFAC là công ty con của ?", "VIC")
]

for question, expected_answer in test_pairs:
    response = query_engine.query(question)
    nodes = retriever.retrieve(question)
    print("Retrieved nodes:")
    for node in nodes:
        print(f"Node: {node.text}")
    print(f"Question: {question}")
    print(f"Expected Answer: {expected_answer}")
    print(f"Actual Answer: {response.response}")
    print()

Retrieved nodes:
Question: lãi sau thuế của Chứng khoán LPBank ?
Expected Answer: Tăng 275% so với cùng kỳ năm trước
Actual Answer: Empty Response

Retrieved nodes:
Question: thu gom cố phiếu MSN?
Expected Answer: Khối ngoại cũng giải ngân mua ròng 179 tỷ tại cổ phiếu MSN
Actual Answer: Empty Response

Retrieved nodes:
Question: Doanh thu Bia Sài Gòn - Quảng Ngãi (BSQ) ?
Expected Answer: Bia Sài Gòn - Quảng Ngãi (BSQ) ghi nhận doanh thu 418 tỷ đồng
Actual Answer: Empty Response

Retrieved nodes:
Node: Here are some facts extracted from the provided text:

Lực mua ròng -> GHI_NHẬN_TẠI -> TCB
Lực mua ròng -> GHI_NHẬN_TẠI -> TPB
Lực mua ròng -> GHI_NHẬN_TẠI -> ACV
Lực mua ròng -> TRONG -> 5 phiên tuần qua
Lực mua ròng -> GHI_NHẬN_TẠI -> MWG
Lực mua ròng -> GHI_NHẬN_TẠI -> YEG

Khối Ngoại Mua Ròng Tại Một Số Cổ Phiếu

Chiều ngược lại, cổ phiếu STB tuần này tiếp tục thu hút dòng vốn khối ngoại chảy vào, được mua ròng mạnh nhất với 221 tỷ đồng. Khối ngoại cũng giải ngân mua ròng 179 tỷ tại c

## Kết luận

Mặc dù liệt kê được nhiều entity và relation từ text, nhưng kết quả query thường trả về empty set. Có vẻ cách trích xuất này chưa hiệu quả lắm.